# Invoice Reconciliation Daily Stats

In [ ]:
import os
from datetime import datetime, timedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill

e.g.
param1 = "some_value"

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql
select now() AT TIME ZONE 'america/vancouver' as current_date

Daily query for details data.

In [ ]:
%%sql daily_reconciliation_details  <<
SELECT i.created_on, i.updated_on, i.id, ir.invoice_number, (i.total - i.service_fees) AS subtotal, i.service_fees, i.total, i.disbursement_status_code, i.payment_method_code, i.invoice_status_code, i.corp_type_code
FROM invoices i join invoice_references ir on ir.invoice_id = i.id
WHERE i.corp_type_code = :partner_code
AND i.invoice_status_code IN ('PAID', 'REFUNDED', 'CANCELLED', 'CREDITED')
AND i.payment_method_code IN ('PAD','EJV', 'DRAWDOWN')
AND date(i.created_on at time zone 'utc' at time zone 'america/vancouver')  > date(current_date - 1 - interval '1 days')
AND date(i.created_on at time zone 'utc' at time zone 'america/vancouver')  <= date(current_date - 1)
ORDER BY 1;

Save to CSV

In [ ]:
filename = os.path.join(os.getcwd(), r'data/')+partner_code+'_daily_reconciliation_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'
df = daily_reconciliation_details.DataFrame()

with open(filename, 'w') as f:
    f.write('Daily Reconciliation Details:\n\n')
    if df.empty:
        f.write('No Data Retrieved')
    else:
        df.to_csv(f, sep=',', encoding='utf-8', index=False)
